In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
locations = ['Chão da Ribeira', 'Sexial', 'Funchal']

In [3]:
def contains_number(input_string):
    for char in input_string:
        if char.isdigit():
            return True
    return False

def parse_title(title): 
    parsed_title = {}

    # todo: optimize
    # todo: handle case when there are several locations in title
    # todo: move to separated fn
    for location in locations: 
        if location in title: 
            parsed_title['location'] = location
            break
        else:
            parsed_title['location'] = '-'

    title_arr = title.split('–')
    parsed_title["title"] = title_arr[0]
    # solve problem with different types of separators
    if len(title_arr) > 1: 
        for arr_elem in title_arr[1:]:
            if contains_number(arr_elem):
                parsed_title["date"] = arr_elem
            else:
                parsed_title["date"] = '-'
    else:
        parsed_title["date"] = '-'
    return parsed_title

In [4]:
url = 'https://www.madeira-web.com/en/whats-on/madeira-events.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')


In [5]:
fields = ["title", "date",  "location", ]

data = {field: [] for field in fields}

In [6]:
jan_h3 = soup.find('h3', id='jan')
feb_h3 = soup.find('h3', id='feb')

jan_to_feb_paragraphs = []

current_tag = jan_h3.find_next(['p', 'h3'])  
while current_tag and current_tag != feb_h3: 
    jan_to_feb_paragraphs.append(current_tag)
    current_tag = current_tag.find_next(['p', 'h3'])

for paragraph in jan_to_feb_paragraphs:
    title_data = parse_title(paragraph.find('strong').text)
    data['title'].append(title_data['title'])
    # todo: add month name to date 
    data['date'].append(title_data['date'])
    data['location'].append(title_data['location'])


In [7]:
df = pd.DataFrame(data)

df

,title,date,location
0,Madeira Plunge,January 1st,-
1,Madeira Orienteering Festival (MOF) 2025,"10-12, 2025",-
2,Madeira Orienteering Meeting (MOM) 2025,-,-
3,Funchal Marathon,-,Funchal
4,Poncha & Honey Exhibition (Mostra da Poncha e ...,-,-
5,"Panelo Festival - Chão da Ribeira, Seixal",-,Chão da Ribeira
